# Naive Bayes Classifiers

## Introduction

Naive Bayes is a class of simple classifiers based on Bayes' Rule and strong (or naive) independence assumptions between features. In this problem, you will implement a Naive Bayes Classifier for the Census Income Data Set from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/).

## Dataset Description

The dataset consists 32561 instances, each representing an individual. The goal is to predict whether a person makes over 50K a year based on 14 features. The features are:

| column | type | description |
| --- |:---:|:--- |
| age | continuous | trips around the sun to date
| final_weight | continuous | census weight attribute; constructed from the original census data |
| education_num | continuous | numeric education scale -- their maximum educational level as a number |
| capital_gain | continuous | income from investment sources |
| capital_loss | continuous | losses from investment sources |
| hours_per_week | continuous | number of hours worked every week |
| work_class | categorical | `Private`, `Self-emp-not-inc`, `Self-emp-inc`, `Federal-gov`, `Local-gov`, `State-gov`, `Without-pay`, `Never-worked` |
| education | categorical | `Bachelors`, `Some-college`, `11th`, `HS-grad`, `Prof-school`, `Assoc-acdm`, `Assoc-voc`, `9th`, `7th-8th`, `12th`, `Masters`, `1st-4th`, `10th`, `Doctorate`, `5th-6th`, `Preschool` |
| marital_status | categorical | `Married-civ-spouse`, `Divorced`, `Never-married`, `Separated`, `Widowed`, `Married-spouse-absent`, `Married-AF-spouse` |
| occupation | categorical | `Tech-support`, `Craft-repair`, `Other-service`, `Sales`, `Exec-managerial`, `Prof-specialty`, `Handlers-cleaners`, `Machine-op-inspct`, `Adm-clerical`, `Farming-fishing`, `Transport-moving`, `Priv-house-serv`, `Protective-serv`, `Armed-Forces` |
| relationship | categorical | `Wife`, `Own-child`, `Husband`, `Not-in-family`, `Other-relative`, `Unmarried.` |
| race | categorical | `White`, `Asian-Pac-Islander`, `Amer-Indian-Eskimo`, `Other`, `Black` |
| sex | categorical | `Female`, `Male` |
| native_country | categorical | (41 values not shown here) |

In [44]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

import gzip
from testing.testing import test

## Q1. Data Preparation

First, you need to load in the above data, provided to you as a CSV file. As the data is from UCI repository, it is already quite clean. However, some instances contain missing `occupation`, `native_country` or `work_class` (represented as ? in the CSV file) and these have to be discarded from the training set. Also, replace the `income` column with `label`, which is 1 if `income` is `>50K` and 0 otherwise. Finally, ensure you reset the index so the row numbers are contiguous.

In [45]:
def read_csv(fn):
    with gzip.open(fn, "rt", newline='', encoding="UTF-8") as file:
        return pd.read_csv(file)

def load_data_test(load_data):
    df = load_data()
    
    DF_TYPES = {
        "age"  : "int64",
        "work_class"  : "object",
        "final_weight"  : "int64",
        "education"  : "object",
        "education_num"  : "int64",
        "marital_status"  : "object",
        "occupation"  : "object",
        "relationship"  : "object",
        "race"  : "object",
        "sex"  : "object",
        "capital_gain"  : "int64",
        "capital_loss"  : "int64",
        "hours_per_week"  : "int64",
        "native_country"  : "object",
        "label"  : "int64"
    }

    test.equal(DF_TYPES, { k: str(df[k].dtypes) for k in DF_TYPES })

    # Check for blank entries:
    test.equal(any(df['occupation'].eq("?")), False)
    test.equal(any(df['native_country'].eq("?")), False)
    test.equal(any(df['work_class'].eq("?")), False)
    
    # Make sure there's no income column:
    test.true("income" not in df.columns)

    # Index handling:
    test.equal(repr(df.index), "RangeIndex(start=0, stop=30162, step=1)")
    
@test
def load_data(file_name="census.csv.gz"):
    """ loads and processes data in the manner specified above


    args:
        file_name : str -- path to csv file containing data

    returns: pd.DataFrame -- processed dataframe
    """
    df = pd.read_csv(file_name)
    df['label'] = 0
    dropped = []
    for i, row in df.iterrows():
        if(row['occupation'] == '?' or row['native_country'] == '?' or row['work_class'] == '?'):
            dropped.append(i)
        elif(row['income'] == '>50K'):
            df.loc[i,'label'] = 1
#             df['label'][i] = 1

    df = df.drop(index = dropped)    
    del df['income']
    df = df.reset_index(drop=True)
#     print(df)
    return df


### TESTING load_data: PASSED 6/6
###



## Overview of Naive Bayes classifier

Let $X_1, X_2, \ldots, X_k$ be the $k$ features of a dataset, with class label given by the variable $y$. A probabilistic classifier assigns the most probable class to each instance $(x_1,\ldots,x_k)$, as expressed by
$$ \hat{y} = \arg\max_y P(y\ \mid\ x_1,\ldots,x_k) $$

Using Bayes' theorem, the above *posterior probability* can be rewritten as
$$ P(y\ \mid\ x_1,\ldots,x_k) = \frac{P(y) P(x_1,\ldots,x_n\ \mid\ y)}{P(x_1,\ldots,x_k)} $$
where
- $P(y)$ is the prior probability of the class
- $P(x_1,\ldots,x_k\ \mid\ y)$ is the likelihood of data under a class
- $P(x_1,\ldots,x_k)$ is the evidence for data

Naive Bayes classifiers assume that the feature values are conditionally independent given the class label, that is,
$ P(x_1,\ldots,x_n\ \mid\ y) = \prod_{i=1}^{k}P(x_i\ \mid\ y) $. This strong assumption helps simplify the expression for posterior probability to
$$ P(y\ \mid\ x_1,\ldots,x_k) = \frac{P(y) \prod_{i=1}^{k}P(x_i\ \mid\ y)}{P(x_1,\ldots,x_k)} $$

For a given input $(x_1,\ldots,x_k)$, $P(x_1,\ldots,x_k)$ is constant. Hence, we can say that:
$$ P(y\ \mid\ x_1,\ldots,x_k) \propto P(y) \prod_{i=1}^{k}P(x_i\ \mid\ y) $$

Thus, the class of a new instance can be predicted as:

$$\hat{y} = \arg\max_y P(y) \prod_{i=1}^{k}P(x_i\ \mid\ y)$$

where $P(y)$ is commonly known as the **class prior** and $P(x_i\ \mid\ y)$ is the **feature predictor**.

Observe that this is the product of $k+1$ probability values, which can result in very small numbers. When working with real-world data, this often leads to an [arithmetic underflow](https://en.wikipedia.org/wiki/Arithmetic_underflow). We will instead be adding the logarithm of the probabilities:

$$\hat{y} = \arg\max_y \underbrace{\log P(y)}_\text{log-prior} + \underbrace{\sum_{i=1}^{k} \log P(x_i\ \mid\ y)}_\text{log-likelihood}$$

The rest of the assignment deals with how each of these probability distributions -- $P(y), P(x_1\ \mid\ y), \ldots, P(x_k\ \mid\ y)$ -- are estimated from data.


### Feature Predictor

Naive Bayes classifiers are popular because we can independently model each feature and mix-and-match model types based on the prior knowledge. For example, we might know (or assume) that $(X_i|y)$ has some distribution, so we can directly use the probability density or mass function of the distribution to model $(X_i|y)$.

In this assignment, you will be using two classes of likelihood models:
- Gaussian models, for continuous real-valued features (parameterized by mean $\mu$ and variance $\sigma$)
- Categorical models, for features in discrete categories (parameterized by $\mathbf{p} = <p_0,p_1\ldots>$, one parameter per category)

You need to implement a generic predictor class for each type of model. Your class should have the following methods:

- `fit()`: Learn parameters for the likelihood model using an appropriate Maximum Likelihood Estimator.
- `partial_log_likelihood()`: Use the previously learnt parameters to compute the probability density or mass of a given feature value, and return the natural logarithm of this value.

## Q2. Gaussian Feature Predictor

The Gaussian distribution is characterized by two parameters - mean $\mu$ and standard deviation $\sigma$:
$$ f_Z(z) = \frac{1}{\sqrt{2\pi}\sigma} \exp{(-\frac{(z-\mu)^2}{2\sigma^2})} $$

Given $n$ samples $z_1, \ldots, z_n$ from the above distribution, the MLE for mean and standard deviation are:
$$ \hat{\mu} = \frac{1}{n} \sum_{j=1}^{n} z_j $$

$$ \hat{\sigma} = \sqrt{\frac{1}{n} \sum_{j=1}^{n} (z_j-\hat{\mu})^2} $$

`scipy.stats.norm` may be helpful, as may `pandas.DataFrame.var`. If you use the latter, remember to correctly set the `ddof=0`!

In [118]:
def gaussian_pred_test(gaussian_predictor):
    g = gaussian_predictor(2)
    
    np.random.seed(0xDEADBEEF)
    rnd = np.random.normal(loc=0.0, scale=1.0, size=(1000,))

    data = pd.Series(np.concatenate([rnd, 100-rnd]))
    labels = pd.Series(np.array([0]*1000 + [1]*1000))

    g.fit(data, labels)

    test.equal(str(type(g.mu)), "<class 'numpy.ndarray'>")
    test.true(np.allclose(g.mu, [0, 100], atol=.1))
    test.equal(str(type(g.sigma)), "<class 'numpy.ndarray'>")
    test.true(np.allclose(g.sigma, [1, 1], atol=.1))

    test.equal(tuple(g.partial_log_likelihood([0., 50., 100.]).shape), (2, 3))
    # If the equality is not exact, you may need to change the test to ensure the absolute difference is no more than 1e-4
    test.true(np.allclose(g.partial_log_likelihood([0., 50., 100.]), [[-0.9234573135702573, -1242.233086628376, -4963.217354198167], [-4963.217354198166, -1242.2330866283753, -0.9234573135702564]], rtol=0, atol=1e-4))

class GaussianPredictor:
    """ Feature predictor for a normally distributed real-valued, continuous feature.

        attr:
            k : int -- number of classes
            mu : np.ndarray[k] -- vector containing per class mean of the feature
            sigma : np.ndarray[k] -- vector containing per class std. deviation of the feature
    """
    
    def __init__(self, k):
        """ constructor

        args : k -- number of classes
        """
        self.k = k

    def fit(self, x, y):
        """update predictor statistics (mu, sigma) for Gaussian distribution

        args:
            x : pd.Series -- feature values
            y : np.Series -- class labels
            
        return : GaussianPredictor -- return self for convenience
        """
        n = len(x)
        classes = list(set(y))
        count = np.zeros(self.k)

        mu = np.zeros(self.k)
        sigma = np.zeros(self.k)
        
        for i in range(n):
            mu[classes.index(y[i])] += x[i]
            count[classes.index(y[i])] += 1
        self.mu = mu/count
        
        for i in range(n):
            sigma[classes.index(y[i])] += (x[i] - self.mu[classes.index(y[i])])**2
        self.sigma = np.sqrt(sigma/count)
#         print(self.mu)
#         print(self.sigma)
        return self
            
    def partial_log_likelihood(self, x):
        """ log likelihood of feature values x according to each class

        args:
            x : pd.Series -- feature values

        return: np.ndarray[self.k, len(x)] : log likelihood for this feature for each class
        """

        result = np.zeros((self.k, len(x)))
        for i,value in enumerate(x):
            result[:, i] = stats.norm.logpdf(np.ones(self.k) * value, loc = self.mu, scale = self.sigma)
        return result

@test
def gaussian_pred(k):
    return GaussianPredictor(k)

### TESTING gaussian_pred: PASSED 6/6
###



## Q3. Categorical Feature Predictor

The categorical distribution with $l$ possible values $\{\text A, \text B, \text C, \ldots\}$ is characterized by the probability distribution $\mathbf p$ over these values. ($\mathbf{p} = (p_0,\dots,p_{l-1})$ where $\sum\mathbf p = 1$.)

If $C$ is categorically distributed, the probability of observing a particular value $z$ is:

$$ \Pr(C=z; \mathbf{p}) = \begin{cases}
    p_0 & \text{ if } z=0
\\  p_1 & \text{ if } z=1
\\  \vdots
\\  p_{l-1} & \text{ if } z=(l-1)
\end{cases}$$

Given $n$ samples $z_1, \ldots, z_n$ from $C$, the smoothed Maximum Likelihood Estimator for $\mathbf p$ is:
$$ \hat{p_t} = \frac{\sum_{j=1}^{n} [z_j=t] + \alpha}{n + l\alpha} $$

The term in the numerator $\sum_{j=1}^{n} [z_j=t]$ is the number of times the value $t$ occurred in the sample. The smoothing is done over all possible values that may be generated by $C$. (This avoids the zero-count problem, similar to the smooting done in $n$-gram language models.)

In this problem, you need to write a predictor that learns a different categorical distribution $C_i$ for each of $k$ possible classes.

#### Specification

  1. You should maintain a dictionary from each possible input token (i.e. each value) to an array of length $k$ that contains $(\Pr(C_0=z), \Pr(C_1=z), ..., \Pr(C_{k-1}=z))$.
  2. For the purpose of smoothing, you should assume that all distributions can produce each value. That is, the set of possible values is the same for all $C_i$.

In [151]:
def categorical_pred_test(categorical_pred):
    # Test One:
    p = categorical_pred(3)
    
    data = pd.Series(["A"]*99 + ["B"]*99 + ["C"]*99)
    labels = pd.Series([0]*99 + [1]*99 + [2]*99)
    p.fit(data, labels)
    
    test.true(np.allclose(p.p['A'], [0.98039216, 0.00980392, 0.00980392], atol=1e-6))

    pll = p.partial_log_likelihood(["A", "B", "C", "A", "B", "C"])
    test.equal(tuple(pll.shape), (3, 6))
    n = np.log(1/102)
    p = np.log(100/102)
    
    test.true(np.allclose(pll, [[p, n, n, p, n, n], [n, p, n, n, p, n], [n, n, p, n, n, p]]))
    
    # Test Two:
    
    p = categorical_pred(2)
    
    data = pd.Series(["A"]*50 + ["B"]*50 + ["C"]*50)
    labels = pd.Series([0]*75 + [1]*75)
    p.fit(data, labels)
    
    test.true(np.allclose(p.p['A'], [0.65384614, 0.01282051], atol=1e-6))
    test.true(np.allclose(p.p['B'], [0.33333334, 0.33333334], atol=1e-6))
    test.true(np.allclose(p.p['C'], [0.01282051, 0.65384614], atol=1e-6))

    pll = p.partial_log_likelihood(["A", "B", "C"])
    test.equal(tuple(pll.shape), (2, 3))
    n = np.log(1/78)
    m = np.log(51/78)
    l = np.log(26/78)

    test.true(np.allclose(pll, [[m, l, n], [n, l, m]], atol=1e-6))
#     If test two fails but test one passes, check the smoothing term in the denominator!

class CategoricalPredictor:
    """ Feature predictor for a categorical feature.

        attr: 
            k : int -- number of classes
            p : Dict[feature_value, np.ndarray[k]] -- dictionary of vectors containing per-class probability of a feature value;
    """
    
    def __init__(self, k):
        """ constructor

        args : k -- number of classes
        """
        self.k = k

    def fit(self, x, y, alpha=1.):
        """ initializes the predictor statistics (p) for Categorical distribution
        
        args:
            x : pd.Series -- feature values
            y : pd.Series -- class labels
        
        kwargs:
            alpha : float -- smoothing factor

        return : CategoricalPredictor -- returns self for convenience
        """
        n = len(x)
        c_x = list(set(x))
        c_y = list(set(y))

        count = np.zeros(self.k)
        p = np.zeros((len(c_x),self.k))
        
        for i in range(n):
            p[c_x.index(x[i]),c_y.index(y[i])] += 1
            count[c_y.index(y[i])] += 1
        p = (p + alpha)/(count + len(c_x)*alpha)
        
        result = {}
        for i,prob in enumerate(p):
            result[c_x[i]] = prob
        self.p = result
        
        return self

    def partial_log_likelihood(self, x):
        """ log likelihood of feature values x according to each class

        args:
            x : pd.Series -- vector of feature values

        return : np.ndarray[self.k, len(x)] -- matrix of log likelihood for this feature
        """
        result = np.zeros((self.k, len(x)))
        for i,value in enumerate(x):
            result[:,i] = np.log(self.p[value])
        return result

@test
def categorical_pred(k):
    return CategoricalPredictor(k)

### TESTING categorical_pred: PASSED 8/8
###



## Q4. Putting things together

It's time to put all the feature predictors together and do something useful! You will implement a class that puts these classifiers to good use:

- `__init__()`: Compute the log prior for each class and initialize the feature predictors (based on feature type). The smoothed prior for class $t$ is given by
$$ \text{prior}(t) = \frac{n_t + \alpha}{n + k\alpha} $$
where $n_t = \sum_{j=1}^{n} [y_j=t]$, (i.e., the number of times the label $t$ occurred in the sample), $n$ is the number fo entries in the sample, and $k$ is the number of label values. 
- `log_likelihood()`: Compute the sum of the log prior and partial log likelihoods for all features. Use it to predict the final class label.
- `predict()`: Use the output of log_likelihood to predict a class label; break ties by predicting the class with lower id.

**Note:** Your implementation should not assume the data will always be the same as the census data. We may pass any dataset to your class. You can assume that:

1. the input will contain a `label` column of type `int64` with values $0,\ldots,k-1$ for some $k$
2. all other columns will be either of type `object` (for categorical data) or `int64` (for integer data)
3. if you encounter a column of an invalid type, throw an exception

In [225]:
def naive_bayes_test(naive_bayes):
    df = load_data()
    cl = naive_bayes(df)
    test.equal(cl.log_prior.tolist(), [-0.28626858222129903, -1.3905468592226538])

    test.true(isinstance(cl.predictor['age'], GaussianPredictor))
    test.true(isinstance(cl.predictor['work_class'], CategoricalPredictor))
    test.true(isinstance(cl.predictor['final_weight'], GaussianPredictor))
    test.true(isinstance(cl.predictor['education'], CategoricalPredictor))
    test.true(isinstance(cl.predictor['education_num'], GaussianPredictor))
    test.true(isinstance(cl.predictor['marital_status'], CategoricalPredictor))
    test.true(isinstance(cl.predictor['occupation'], CategoricalPredictor))
    test.true(isinstance(cl.predictor['relationship'], CategoricalPredictor))
    test.true(isinstance(cl.predictor['race'], CategoricalPredictor))
    test.true(isinstance(cl.predictor['sex'], CategoricalPredictor))
    test.true(isinstance(cl.predictor['capital_gain'], GaussianPredictor))
    test.true(isinstance(cl.predictor['capital_loss'], GaussianPredictor))
    test.true(isinstance(cl.predictor['hours_per_week'], GaussianPredictor))
    test.true(isinstance(cl.predictor['native_country'], CategoricalPredictor))    

    ll = cl.log_likelihood(df.drop("label", axis="columns"))
    test.equal(tuple(ll.shape), (2, 30162))
    test.equal(ll[:,:2].tolist(), [[-49.84977999441486, -50.38520793711001], [-53.407383777033196, -51.30832341372758]])

    lp = cl.predict(df.drop("label", axis="columns"))
    test.equal(tuple(lp.shape), (30162,))
    test.equal(sum(lp), 5407)
    test.equal(lp[:10].tolist(), [0]*8 + [1]*2)
    
#     # Make sure you can handle a non-contiguous index, pt. 1:
#     cl = naive_bayes(df[1::3])
#     ll = cl.log_likelihood(df.drop("label", axis="columns"))
#     test.equal(tuple(ll.shape), (2, 30162))


#     # Make sure you can handle a non-contiguous index, pt. 2:
#     tdf = df.drop(["age", "sex", "work_class"], axis="columns")
#     cl = naive_bayes(tdf[1::2])
#     ll = cl.log_likelihood(tdf.drop("label", axis="columns"))
#     test.equal(tuple(ll.shape), (2, 30162))

class NaiveBayesClassifier:
    """ Naive Bayes classifier for a mixture of continuous and categorical attributes.
        We use GaussianPredictor for continuous attributes and CategoricalPredictor for categorical ones.
        
        attr:
            predictor : Dict[column_name,model] -- model for each column
            log_prior : np.ndarray -- the (log) prior probability of each class
    """

    def __init__(self, df, alpha=1.):
        """initialize predictors for each feature and compute class prior
        
        args:
            df : pd.DataFrame -- processed dataframe, without any missing values.
        
        kwargs:
            alpha : float -- smoothing factor for prior probability
        """
        predictor = {}
        k = len(set(df['label']))
        self.k = k
        for title in df.drop("label", axis="columns"):
#             print(df[title].dtype)
            if (df[title].dtype == "int64"):
                predictor[title] = GaussianPredictor(k)
                predictor[title].fit(list(df[title]),list(df['label']))
            elif (df[title].dtype == "object"):
                predictor[title] = CategoricalPredictor(k)
                predictor[title].fit(list(df[title]),list(df['label']))
#             raise ValueError(df[title].dtype)
#         print(predictor)
        self.predictor = predictor
    
        count = np.zeros(k)
        classes = list(set(df['label']))
        df = df.reset_index(drop=True)
        n = len(df['label'])
        for i in range(n):
#             print(df['label'])
#             print(classes.index(df['label'][i]))
            count[classes.index(df['label'][i])] += 1
        self.log_prior = np.log((count + alpha)/(n + k * alpha))

    def log_likelihood(self, x):
        """log_likelihood for input instances from log_prior and partial_log_likelihood of feature predictors

        args:
            x : pd.DataFrame -- processed dataframe (ignore label if present)

        returns : np.ndarray[num_classes, len(x)] -- array of log-likelihood
        """
        
        ll = np.zeros((self.k, len(x)))
#         print(len(x))
        for i in range(len(x)):
            ll[:,i] = self.log_prior
#         for title in x:
#             print(title)
        for title in x:
#             print(title)
#             print(self.predictor[title])
            if(title != "label"):
                ll += self.predictor[title].partial_log_likelihood(list(x[title]))
#         print(self.predictor['age'].partial_log_likelihood(list(df['age'])))
        return ll

    def predict(self, x):
        """predicts label for input instances, breaks ties in favor of the class with lower id.

        args:
            x : pd.DataFrame -- processed dataframe (ignore label if present)

        returns : np.ndarray[len(x)] -- vector of class labels
        """
        prediction = np.zeros(len(x))
        ll = self.log_likelihood(x)
        labels = ll.argmax(axis = 0)
        return labels

@test
def naive_bayes(*args, **kwargs):
    return NaiveBayesClassifier(*args, **kwargs)

### TESTING naive_bayes: PASSED 19/20
# 16	: Failed: [[-49.84977999441482, -50.38520793710976], [-53.40738377703323, -51.30832341372785]] is not equal to [[-49.84977999441486, -50.38520793711001], [-53.407383777033196, -51.30832341372758]]
###



In [154]:
# df = load_data()

In [186]:
# k = len(set(df['label']))
# predictor = {}
# for title in df:
# #     print(title)
#     if (df[title].dtype == "int64"):
#         predictor[title] = GaussianPredictor(k)
#         predictor[title].fit(df[title],df['label'])
#     elif (df[title].dtype == "object"):
#         predictor[title] = CategoricalPredictor(k)
#         predictor[title].fit(df[title],df['label'])
# # self.predictor = predictor
# # print(predictor)
# print(len(set(df["label"])))

2


In [194]:
# count = np.zeros(2)
# classes = list(set(df['label']))

# n = len(df['label'])
# for i in range(n):
#     count[classes.index(df['label'][i])] += 1
# log_prior = np.log((count + 1)/(n + 2 * 1))
# print(log_prior)

[-0.28626858 -1.39054686]


In [193]:
# x = df.drop("label", axis="columns")
# # print(x)
# # cl = naive_bayes(df)
# a = predictor['age'].partial_log_likelihood(list(df['age']))
# print(a.shape)

(2, 30162)
